# Interpolate all quantities on rectilinear grid and save to one vtk file

# Imports

In [7]:
import numpy as np
from scipy.interpolate import griddata

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

import sys

In [8]:
from pyvisfile.vtk import write_structured_grid
from pytools.obj_array import make_obj_array

In [9]:
sys.path.append('../../read_from_sql/')
import read_from_sql
sys.path.append('/Users/vonderlinden2/rsx_analysis/mach_probe_analysis')
sys.path.append('/Users/vonderlinden2/rsx_analysis/time_alignment/source/')
import ion_current_to_mach_number as ic_to_mach
reload(ic_to_mach)
sys.path.append('/Users/vonderlinden2/rsx_analysis/time_alignment/source')
import absolute_times as at

import structured_3d_vtk as struc_3d
reload(struc_3d)

<module 'structured_3d_vtk' from 'structured_3d_vtk.pyc'>

# Make full grid

In [10]:
spatial_increment = 0.001
x_min, x_max = -0.027, 0.024
y_min, y_max = -0.021, 0.03
z_min, z_max = 0.249, 0.416
full_grid_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

full_grid, sizes = struc_3d.bounded_grid(full_grid_bounds, spatial_increment)
full_vtk_grid = struc_3d.prepare_mesh(full_grid, sizes)

In [11]:
full_vtk_grid.shape

(3, 51, 51, 167)

# Magnetic Field 

## first make sub-grid

In [12]:
x_min, x_max = -0.027, 0.022
y_min, y_max = -0.021, 0.0295 
z_min, z_max = 0.249, 0.416

b_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

b_indices = struc_3d.build_sub_grid_indices(full_vtk_grid, b_bounds)

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

for plane in [0.249, 0.302, 0.357, 0.416]:
    bx_measurements[plane] = struc_3d.average_duplicate_points(bx_measurements[plane])
    by_measurements[plane] = struc_3d.average_duplicate_points(by_measurements[plane])
    bz_measurements[plane] = struc_3d.average_duplicate_points(bz_measurements[plane])

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 

(b_vtk_grid, 
 b_vtk_vectors) = struc_3d.build_vtk_vector(direction_measurements, 
                                            full_vtk_grid=full_vtk_grid, indices=b_indices)

b_full_vtk_indices = struc_3d.build_vtk_sub_indices(b_indices)

b_full_vtk_vectors = []
for b_vtk_vector in b_vtk_vectors:
    b_x_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    b_y_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    b_z_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    b_x_full_vtk[b_full_vtk_indices[0], b_full_vtk_indices[1], b_full_vtk_indices[2]] = b_vtk_vector[0][0]
    b_y_full_vtk[b_full_vtk_indices[0], b_full_vtk_indices[1], b_full_vtk_indices[2]] = b_vtk_vector[1][0]
    b_z_full_vtk[b_full_vtk_indices[0], b_full_vtk_indices[1], b_full_vtk_indices[2]] = b_vtk_vector[2][0]
    b_full_vtk_vector_x = np.expand_dims(b_x_full_vtk, 0)
    b_full_vtk_vector_y = np.expand_dims(b_y_full_vtk, 0)
    b_full_vtk_vector_z = np.expand_dims(b_z_full_vtk, 0)
    b_full_vtk_vector = make_obj_array([b_full_vtk_vector_x, b_full_vtk_vector_y, b_full_vtk_vector_z])
    b_full_vtk_vectors.append(b_full_vtk_vector)

time_point 0
time_point 1
time_point 2
time_point 3
time_point 4
time_point 5
time_point 6
time_point 7
time_point 8
time_point 9
time_point 10
time_point 11
time_point 12
time_point 13
time_point 14
time_point 15
time_point 16
time_point 17
time_point 18
time_point 19
time_point 20


# Temperature & density

In [13]:
x_min, x_max = -0.022, 0.018
y_min, y_max = -0.021, 0.0255 
z_min, z_max = 0.249, 0.416

tp_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

tp_indices = struc_3d.build_sub_grid_indices(full_vtk_grid, tp_bounds)

te_measurements = struc_3d.read_idl('te')
n_measurements = struc_3d.read_idl('n')

for plane in [0.249, 0.302, 0.357, 0.416]:
    te_measurements[plane] = struc_3d.average_duplicate_points(te_measurements[plane])
    n_measurements[plane] = struc_3d.average_duplicate_points(n_measurements[plane])

(te_vtk_grid, 
 te_vtk_scalars) = struc_3d.build_vtk_scalar(te_measurements, 
                                             full_vtk_grid=full_vtk_grid, indices=tp_indices)

(n_vtk_grid, 
 n_vtk_scalars) = struc_3d.build_vtk_scalar(n_measurements, 
                                            full_vtk_grid=full_vtk_grid, indices=tp_indices)

tp_full_vtk_indices = struc_3d.build_vtk_sub_indices(tp_indices)

te_full_vtk_scalars = []
n_full_vtk_scalars = []
for te_vtk_scalar in te_vtk_scalars:
    te_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    te_full_vtk[tp_full_vtk_indices[0], tp_full_vtk_indices[1], tp_full_vtk_indices[2]] = te_vtk_scalar[0]
    te_full_vtk_scalar = np.expand_dims(te_full_vtk, 0)
    te_full_vtk_scalars.append(te_full_vtk_scalar)
    
for n_vtk_scalar in n_vtk_scalars:
    n_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    n_full_vtk[tp_full_vtk_indices[0], tp_full_vtk_indices[1], tp_full_vtk_indices[2]] = n_vtk_scalar[0]
    n_full_vtk_scalar = np.expand_dims(n_full_vtk, 0)
    n_full_vtk_scalars.append(n_full_vtk_scalar)

structured_3d_vtk.py:253: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  assert (((not full_vtk_grid == None and not indices == None) or
structured_3d_vtk.py:255: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  not (not full_vtk_grid == None and not indices == None and
structured_3d_vtk.py:257: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if not full_vtk_grid == None and not indices == None:


time_point 0
time_point 1
time_point 2
time_point 3
time_point 4
time_point 5
time_point 6
time_point 7
time_point 8
time_point 9
time_point 10
time_point 11
time_point 12
time_point 13
time_point 14
time_point 15
time_point 16
time_point 17
time_point 18
time_point 19
time_point 20
time_point 0
time_point 1
time_point 2
time_point 3
time_point 4
time_point 5
time_point 6
time_point 7
time_point 8
time_point 9
time_point 10
time_point 11
time_point 12
time_point 13
time_point 14
time_point 15
time_point 16
time_point 17
time_point 18
time_point 19
time_point 20


In [14]:
n_full_vtk_scalars[0].shape

(1, 51, 51, 167)

# Mach number

In [15]:
database = '/Users/vonderlinden2/rsx_analysis/shots_database/source/shots.db'
table = 'Shots'
z_direction_1, z_direction_2 = 0, 180
y_direction_1, y_direction_2 = 90, 270
angle_signs = {0: 1,
               180: -1,
               90: -1,
               0: 1}
min_spectral_density = 1.6e-8

In [16]:
condition_z_0416 = ("campaigns = 'mach_probe_plane_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(z_direction_1) +
                    " OR mach_orientation = " + str(z_direction_2) + ")")

condition_y_0416 = ("campaigns = 'mach_probe_plane_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(y_direction_1) +
                    " OR mach_orientation = " + str(y_direction_2) + ")")

cursor, connection = read_from_sql.cursor_with_rows(condition_z_0416,
                                                    database,
                                                    table)
z_0416_shots = cursor.fetchall()
cursor.close()
connection.close()

cursor, connection = read_from_sql.cursor_with_rows(condition_y_0416,
                                                    database,
                                                    table)
y_0416_shots = cursor.fetchall()
cursor.close()
connection.close()

condition_z_302 = ("campaigns = 'mach_probe_plane_campaign_2'"
                   " AND fiducial_pre_crowbar_gyration_spectral_density > "
                   + str(min_spectral_density) +
                   " AND mach_signals_exist = 1"
                   " AND (mach_orientation = " + str(z_direction_1) +
                   " OR mach_orientation = " + str(z_direction_2) + ")")

cursor, connection = read_from_sql.cursor_with_rows(condition_z_302,
                                                    database,
                                                    table)
z_0302_shots = cursor.fetchall()
cursor.close()
connection.close()

In [18]:
mach_z_0416_measurements = ic_to_mach.run_mach_analysis(z_0416_shots,
                                                        21, 
                                                        angle_signs) 
mach_y_0416_measurements = ic_to_mach.run_mach_analysis(y_0416_shots,
                                                        21, 
                                                        angle_signs)
mach_z_0302_measurements = ic_to_mach.run_mach_analysis(z_0302_shots,
                                                        21, 
                                                        angle_signs)

mach_z_0416_measurements['delays'] = range(21)
mach_y_0416_measurements['delays'] = range(21)
mach_z_0302_measurements['delays'] = range(21)

mach_z_0416_measurements = struc_3d.average_duplicate_points(mach_z_0416_measurements)
mach_y_0416_measurements = struc_3d.average_duplicate_points(mach_y_0416_measurements)
mach_z_0302_measurements = struc_3d.average_duplicate_points(mach_z_0302_measurements)

mach_y_measurements = {0.416: mach_y_0416_measurements}
mach_z_measurements = {0.302: mach_z_0302_measurements,
                       0.416: mach_z_0416_measurements}

In [19]:
x_min, x_max = -0.027, 0.022
y_min, y_max = -0.02, 0.03
z_min, z_max = 0.416, 0.416
mach_y_0416_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

z_min, z_max = 0.302, 0.416
mach_z_plane_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

mach_y_indices = struc_3d.build_sub_grid_indices(full_vtk_grid, mach_y_0416_bounds)

mach_z_indices = struc_3d.build_sub_grid_indices(full_vtk_grid, mach_z_plane_bounds)

In [20]:
(mach_z_vtk_grid, 
 mach_z_vtk_scalars) = struc_3d.build_vtk_scalar(mach_z_measurements,
                                                full_vtk_grid=full_vtk_grid, 
                                                indices=mach_z_indices,
                                                z_planes=[0.302, 0.416])

(mach_y_vtk_grid, 
 mach_y_vtk_scalars) = struc_3d.build_vtk_scalar(mach_y_measurements,
                                                full_vtk_grid=full_vtk_grid, 
                                                indices=mach_y_indices,
                                                z_planes=[0.416])

time_point 0
time_point 1
time_point 2
time_point 3
time_point 4
time_point 5
time_point 6
time_point 7
time_point 8
time_point 9
time_point 10
time_point 11
time_point 12
time_point 13
time_point 14
time_point 15
time_point 16
time_point 17
time_point 18
time_point 19
time_point 20
time_point 0
time_point 1
time_point 2
time_point 3
time_point 4
time_point 5
time_point 6
time_point 7
time_point 8
time_point 9
time_point 10
time_point 11
time_point 12
time_point 13
time_point 14
time_point 15
time_point 16
time_point 17
time_point 18
time_point 19
time_point 20


In [21]:
mach_y_full_vtk_indices = struc_3d.build_vtk_sub_indices(mach_y_indices)
mach_z_full_vtk_indices = struc_3d.build_vtk_sub_indices(mach_z_indices)

In [22]:
mach_y_full_vtk_indices[0].shape

(49, 50, 1)

In [23]:
mach_full_vtk_vectors = []
for mach_y_vtk_scalar, mach_z_vtk_scalar in zip(mach_y_vtk_scalars, mach_z_vtk_scalars):
    mach_x_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    mach_y_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    mach_z_full_vtk = np.zeros(full_vtk_grid.shape[1:])
    mach_y_full_vtk[mach_y_full_vtk_indices[0], mach_y_full_vtk_indices[1], mach_y_full_vtk_indices[2]] = mach_y_vtk_scalar[0]
    mach_z_full_vtk[mach_z_full_vtk_indices[0], mach_z_full_vtk_indices[1], mach_z_full_vtk_indices[2]] = mach_z_vtk_scalar[0]
    mach_full_vtk_vector_x = np.expand_dims(mach_x_full_vtk, 0)
    mach_full_vtk_vector_y = np.expand_dims(mach_y_full_vtk, 0)
    mach_full_vtk_vector_z = np.expand_dims(mach_z_full_vtk, 0)
    mach_full_vtk_vector = make_obj_array([mach_full_vtk_vector_x, mach_full_vtk_vector_y, mach_full_vtk_vector_z])
    mach_full_vtk_vectors.append(mach_full_vtk_vector)

# Write all data to a single vtk file

In [24]:
output_path = '../output/2016-07-06/'
for time_point in xrange(len(b_full_vtk_vectors)):
    point_data = [('B', b_full_vtk_vectors[time_point]),
                  ('Te', te_full_vtk_scalars[time_point]),
                  ('n', n_full_vtk_scalars[time_point]),
                  ('mach', mach_full_vtk_vectors[time_point])]
    write_structured_grid(output_path + 
                          'all' + str(time_point).zfill(4) + '.vts', full_vtk_grid, point_data=point_data)

In [25]:
for time_point in xrange(len(b_full_vtk_vectors)):
    ending = 'all' + str(time_point).zfill(4) + '.npz'
    np.savez(output_path + ending, B_x=b_full_vtk_vectors[time_point][0], 
                                   B_y=b_full_vtk_vectors[time_point][1],
                                   B_z=b_full_vtk_vectors[time_point][2],
                                   Te=te_full_vtk_scalars[time_point], 
                                   n=n_full_vtk_scalars[time_point],
                                   mach_x=mach_full_vtk_vectors[time_point][0],
                                   mach_y=mach_full_vtk_vectors[time_point][1],
                                   mach_z=mach_full_vtk_vectors[time_point][2],
                                   grid=full_vtk_grid,
                                   b_indices=b_indices,
                                   tp_indices=tp_indices,
                                   mach_y_indices=mach_y_indices,
                                   mach_z_indices=mach_z_indices,
                                   b_grid=b_vtk_grid,
                                   tp_grid=n_vtk_grid,
                                   mach_y_grid=mach_y_vtk_grid,
                                   mach_z_grid=mach_z_vtk_grid)